Ensambles de Modelos --- 12:19 min
===

* 12:19 min | Ultima modificación: Abril 14, 2021 | [YouTube](https://youtu.be/oQYYx9ZMync)

La idea de combinar los resultados de varios modelos no es nueva; esta es conocida en la literatura de redes neuronales como ensambles de modelos, y en estadística como combinación de pronósticos. En esta lección se presentan los fundamentos de esta metodología y algunos de los modelos derivados a partir de esta idea.

## Definición

Un ensamble es un tipo de modelo que permite la combinación de varios modelos de predicción para obtener un solo pronóstico basado en los pronósticos individuales de cada modelo. En la figura siguiente se presenta un esquema ilustrativo.

![alt text](assets/combiner.jpg)

La clave de la operación de esta metodología se basa en la diversidad, la cual puede obtenerse de diferentes formas:

* Variando los datos de entrenamiento: se usa el mismo modelo en todos los casos, pero para cada uno de ellos se usa una muestra de entrenamiento obtenida por boostraping; así cada modelo tiene parámetros diferentes ya que fue estimado sobre una muestra diferente.


* Variando la configuración del modelo: se usan exactamente los mismos datos de entrenamiento, pero sobre diferentes modelos obtenidos variando su configuración; por ejemplo, el mismo modelo pero con diferente configuración (entradas usadas, complejidad, etc). Inclusive se pueden utilizar distintos modelos.


* Una combinación de los dos anteriores.

El combinador es un mecanismo que obtiene un único pronóstico a partir de los pronósticos individuales de cada modelo. Para problemas de clasificación, la conbinación se hace por votación. En problemas de regresión, mediante promedio simple, promedio combinado o, inclusive, regresión.

Diferentes metodologías se han desarrollado sobre este concepto.

**Bagging (Bootstrap aggregation).---** En esta metodología, la diversidad se obtiene al entrenar un mismo modelo sobre diferentes conjuntos de entrenamiento usando bootstraping. La combinación se hace por votación para problemas de clasificación y por promedio para problemas numéricos.

**AdaBoosting (Adaptive Boosting).---**  En este caso, los conjuntos de dato son diseñados especificamente para generar modelos complementarios. De forma simplificada, el algoritmo procede de la siguiente forma:


* Paso 1: Se construye un clasificador sobre todos los datos de entrenamiento.


* Paso 2: Se construye un nuevo conjunto de datos con los ejemplos mal clasificados (o una porción de ellos).


* Paso 3: se construye un nuevo clasificador con los datos obtenidos en el paso 2.


* Paso 4: Se retorna al Paso 2.


El proceso itera hasta que se alcanza una precisión requerida o el número máximo de clasificadores permitidos en el ensamble. La ponderación se realiza dando más peso a los modelos con mejor desempeño, de tal forma, que el desempeño es, al menos, similar al del mejor clasificador obtenido.

**Random forest.---**  En este método se combinan el bagging con la selección aleatoria de caracteríticas para aumetar la diversidad. La salida del modelo se obtiene por votación. Cada ejemplo que no es considerado durante el entrenamiento es usado como parte del conjunto de prueba.

## Carga y preparación

In [1]:
import pandas as pd

#
# Lee el archivo.
#
df = pd.read_csv(
    "https://raw.githubusercontent.com/jdvelasq/datalabs/master/datasets/credit.csv",
    sep=",",
    thousands=None,
    decimal=".",
    encoding="latin-1",
)

#
# Verifica la lectura de los datos
#
df.describe()

,months_loan_duration,amount,installment_rate,residence_history,age,existing_credits,default,dependents
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,20.903000,3271.258000,2.973000,2.845000,35.546000,1.407000,1.300000,1.155000
std,12.058814,2822.736876,1.118715,1.103718,11.375469,0.577654,0.458487,0.362086
min,4.000000,250.000000,1.000000,1.000000,19.000000,1.000000,1.000000,1.000000
25%,12.000000,1365.500000,2.000000,2.000000,27.000000,1.000000,1.000000,1.000000
50%,18.000000,2319.500000,3.000000,3.000000,33.000000,1.000000,1.000000,1.000000
75%,24.000000,3972.250000,4.000000,4.000000,42.000000,2.000000,2.000000,1.000000
max,72.000000,18424.000000,4.000000,4.000000,75.000000,4.000000,2.000000,2.000000


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   checking_balance      1000 non-null   object
 1   months_loan_duration  1000 non-null   int64 
 2   credit_history        1000 non-null   object
 3   purpose               1000 non-null   object
 4   amount                1000 non-null   int64 
 5   savings_balance       1000 non-null   object
 6   employment_length     1000 non-null   object
 7   installment_rate      1000 non-null   int64 
 8   personal_status       1000 non-null   object
 9   other_debtors         1000 non-null   object
 10  residence_history     1000 non-null   int64 
 11  property              1000 non-null   object
 12  age                   1000 non-null   int64 
 13  installment_plan      1000 non-null   object
 14  housing               1000 non-null   object
 15  existing_credits      1000 non-null   i

In [3]:
df.columns

Index(['checking_balance', 'months_loan_duration', 'credit_history', 'purpose',
       'amount', 'savings_balance', 'employment_length', 'installment_rate',
       'personal_status', 'other_debtors', 'residence_history', 'property',
       'age', 'installment_plan', 'housing', 'existing_credits', 'default',
       'dependents', 'telephone', 'foreign_worker', 'job'],
      dtype='object')

In [4]:
#
# Contenido del archivo
#
df.head()

,checking_balance,months_loan_duration,credit_history,purpose,amount,savings_balance,employment_length,installment_rate,personal_status,other_debtors,...,property,age,installment_plan,housing,existing_credits,default,dependents,telephone,foreign_worker,job
0,< 0 DM,6,critical,radio/tv,1169,unknown,> 7 yrs,4,single male,none,...,real estate,67,none,own,2,1,1,yes,yes,skilled employee
1,1 - 200 DM,48,repaid,radio/tv,5951,< 100 DM,1 - 4 yrs,2,female,none,...,real estate,22,none,own,1,2,1,none,yes,skilled employee
2,unknown,12,critical,education,2096,< 100 DM,4 - 7 yrs,2,single male,none,...,real estate,49,none,own,1,1,2,none,yes,unskilled resident
3,< 0 DM,42,repaid,furniture,7882,< 100 DM,4 - 7 yrs,2,single male,guarantor,...,building society savings,45,none,for free,1,1,2,none,yes,skilled employee
4,< 0 DM,24,delayed,car (new),4870,< 100 DM,1 - 4 yrs,3,single male,none,...,unknown/none,53,none,for free,2,2,2,none,yes,skilled employee


### Códificación de variables como factores

In [5]:
from sklearn.preprocessing import LabelEncoder

for column in [
    "checking_balance",
    "credit_history",
    "purpose",
    "savings_balance",
    "employment_length",
    "personal_status",
    "other_debtors",
    "property",
    "installment_plan",
    "housing",
    "telephone",
    "foreign_worker",
    "job",
]:

    enc = LabelEncoder()
    df[column] = enc.fit_transform(df[column])

### Muestras de entrenamiento y prueba

In [6]:
#
#  Se usa el 90% de los datos para entrenamiento
#  y el 10% restante para prueba
#
train_sample = list(range(900))
test_sample = list(range(900, 1000))

In [7]:
#
# Genera los conjuntos de entrenamiento y prueba
#
y_train_true = df.loc[train_sample]["default"]
y_test_true = df.loc[test_sample]["default"]

df.drop("default", axis=1, inplace=True)
X_train = df.loc[train_sample]
X_test = df.loc[test_sample]

### Construcción de modelos y métricas de desempeño

In [8]:
#
# Random Forest
#
from sklearn.ensemble import (
    AdaBoostClassifier,
    GradientBoostingClassifier,
    RandomForestClassifier,
)

y_test_pred_RandomForest = (
    RandomForestClassifier(n_estimators=10).fit(X_train, y_train_true).predict(X_test)
)

y_test_pred_AdaBoost = (
    AdaBoostClassifier(n_estimators=10).fit(X_train, y_train_true).predict(X_test)
)

y_test_pred_GradientBoosting = (
    GradientBoostingClassifier(n_estimators=10)
    .fit(X_train, y_train_true)
    .predict(X_test)
)

In [9]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test_pred_RandomForest, y_test_true)

array([[59, 17],
       [ 9, 15]])

In [10]:
confusion_matrix(y_test_pred_AdaBoost, y_test_true)

array([[60, 19],
       [ 8, 13]])

In [11]:
confusion_matrix(y_test_pred_GradientBoosting, y_test_true)

array([[67, 26],
       [ 1,  6]])